In [2]:
%tensorflow_version 1.x

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.python.lib.io import file_io

%matplotlib inline

import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from sklearn.metrics import *
from keras.engine import Model
from keras.layers import Input, Flatten, Dense, Activation, Conv2D, MaxPool2D, BatchNormalization, Dropout, MaxPooling2D
import skimage
from skimage.transform import rescale, resize

Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
Resize_pixelsize = 197

In [6]:
# Function that reads the data from the csv file, increases the size of the images and returns the images and their labels
    # dataset: Data path
def get_data(dataset):
    
    file_stream = file_io.FileIO(dataset, mode='r')
    data = pd.read_csv(file_stream)
    data['pixels'] = data['pixels'].apply(lambda x: [int(pixel) for pixel in x.split()])
    X, Y = data['pixels'].tolist(), data['emotion'].values
    X = np.array(X, dtype='float32').reshape(-1,48,48,1)
    X = X/255.0
   
    X_res = np.zeros((X.shape[0], Resize_pixelsize,Resize_pixelsize,3))
    for ind in range(X.shape[0]): 
        sample = X[ind]
        sample = sample.reshape(48, 48)
        image_resized = resize(sample, (Resize_pixelsize, Resize_pixelsize), anti_aliasing=True)
        X_res[ind,:,:,:] = image_resized.reshape(Resize_pixelsize,Resize_pixelsize,1)

    Y_res = np.zeros((Y.size, 7))
    Y_res[np.arange(Y.size),Y] = 1    
    
    return  X, X_res, Y_res

In [7]:
dev_dataset_dir = '/content/drive/My Drive/Colab Notebooks/dataset/private_test.csv'
test_dataset_dir = '/content/drive/My Drive/Colab Notebooks/dataset/public_test.csv'

X_dev, X_res_dev, Y_dev   = get_data(dev_dataset_dir)
X_test, X_res_test, Y_test   = get_data(test_dataset_dir)

In [8]:
model = load_model('/content/drive/My Drive/Colab Notebooks/models/SOA-EPOCHS_150-DROPOUT_0.3-test_acc_0.671.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.




In [9]:
print('\n# Evaluate on dev data')
results_dev = model.evaluate(X_dev,Y_dev)
print('dev loss, dev acc:', results_dev)

print('\n# Evaluate on test data')
results_test = model.evaluate(X_test,Y_test)
print('test loss, test acc:', results_test)


# Evaluate on dev data
3589/3589 [==============================] - 8s 2ms/step
dev loss, dev acc: [0.8966232636540207, 0.6701030731201172]

# Evaluate on test data
3589/3589 [==============================] - 0s 139us/step
test loss, test acc: [0.9666584921965994, 0.6497631669044495]


In [10]:
model2 = load_model('/content/drive/My Drive/Colab Notebooks/models/SOA-EPOCHS_150-DROPOUT_0.4-test_acc_0.658.h5')
print('\n# Evaluate on dev data')
results_dev = model2.evaluate(X_dev,Y_dev)
print('dev loss, dev acc:', results_dev)

print('\n# Evaluate on test data')
results_test = model2.evaluate(X_test,Y_test)
print('test loss, test acc:', results_test)


# Evaluate on dev data
3589/3589 [==============================] - 1s 189us/step
dev loss, dev acc: [0.9148297683214138, 0.6572861671447754]

# Evaluate on test data
3589/3589 [==============================] - 0s 139us/step
test loss, test acc: [0.9832127940259494, 0.6425188183784485]


In [11]:
Resnet_model = load_model('/content/drive/My Drive/Colab Notebooks/models/RESNET50-EPOCHS_30test_acc_0.687.h5')
print('\n# Evaluate on dev data')
results_dev = Resnet_model.evaluate(X_res_dev,Y_dev)
print('dev loss, dev acc:', results_dev)

print('\n# Evaluate on test data')
results_test = Resnet_model.evaluate(X_res_test,Y_test)
print('test loss, test acc:', results_test)



# Evaluate on dev data
3589/3589 [==============================] - 13s 4ms/step
dev loss, dev acc: [0.8754108052095619, 0.6884925961494446]

# Evaluate on test data
3589/3589 [==============================] - 12s 3ms/step
test loss, test acc: [0.9242907606416109, 0.669824481010437]


In [13]:
VGG_model = load_model("/content/drive/My Drive/Colab Notebooks/models/VGG16-EPOCHS_20test_acc_0.670.h5")
print('\n# Evaluate on dev data')
results_dev = VGG_model.evaluate(X_res_dev,Y_dev)
print('dev loss, dev acc:', results_dev)

print('\n# Evaluate on test data')
results_test = VGG_model.evaluate(X_res_test,Y_test)
print('test loss, test acc:', results_test)


# Evaluate on dev data
3589/3589 [==============================] - 18s 5ms/step
dev loss, dev acc: [0.8959327053562052, 0.6703817248344421]

# Evaluate on test data
3589/3589 [==============================] - 14s 4ms/step
test loss, test acc: [0.9356246001210907, 0.6586793065071106]


In [14]:
senet_model = load_model("/content/drive/My Drive/Colab Notebooks/models/SENET50-EPOCHS_30test_acc_0.702.h5")
print('\n# Evaluate on dev data')
results_dev = senet_model.evaluate(X_res_dev,Y_dev)
print('dev loss, dev acc:', results_dev)

print('\n# Evaluate on test data')
results_test = senet_model.evaluate(X_res_test,Y_test)
print('test loss, test acc:', results_test)


# Evaluate on dev data
3589/3589 [==============================] - 15s 4ms/step
dev loss, dev acc: [0.9776307813601096, 0.6979659795761108]

# Evaluate on test data
3589/3589 [==============================] - 13s 4ms/step
test loss, test acc: [1.0536655107392146, 0.6765115857124329]


In [15]:
models_SOA = [model, model2]
models_TL = [Resnet_model, senet_model,  VGG_model]

In [16]:
def ensemble_predictions(models_SOA, testX, models_TL, testresX):
  # make predictions
  yhats = np.zeros((len(models_SOA)+len(models_TL),testX.shape[0],7))

  for model_ind in range(len(models_SOA)):
    yhat = models_SOA[model_ind].predict(testX)
    yhats[model_ind,:,:] = yhat

  for model_ind in range(len(models_TL)):
    yhat = models_TL[model_ind].predict(testresX)
    yhats[len(models_SOA)+model_ind,:,:] = yhat

  summed = np.sum(yhats, axis=0)
  result = np.argmax(summed, axis=1)
  return result
 
# evaluate a specific number of members in an ensemble
def evaluate_n_members(models_SOA, testX, models_TL, testresX, testy):
	# select a subset of members
	#subset = members[:n_members]
	#print(len(subset))
	# make prediction
	yhat = ensemble_predictions(models_SOA, testX, models_TL, testresX)
	# calculate accuracy
	return accuracy_score(testy, yhat)

In [17]:
ens_acc = evaluate_n_members(models_SOA, X_test, models_TL, X_res_test, np.argmax(Y_test, axis=1))
print(ens_acc)

0.7118974644747841
